In [1]:
import requests
import threading
import pandas as pd
import flask
from flask import Flask, request
import geopy
from geopy import Nominatim
from translate import Translator
from langdetect import detect
from iso639 import languages
from textblob import TextBlob

In [2]:
key = 'fsq3bR9nCSrR/WbzD82rlvh990Q70wuc8BuuRs0Ypm6fx+w='

# this method helps us get long and lat of certain city

def get_latlong(b):

    
    try:
    
        city = b

        geolocator = Nominatim(user_agent = 'Safra')
    
        loc = geolocator.geocode(city)
         
        # by default
    except: return 'No results found' , f'{b}'
    
    return loc.latitude , loc.longitude

In [3]:
# this is a test
lat , long = get_latlong('jeddah')

In [4]:
lat , long

(21.5810088, 39.1653612)

In [5]:
def translate(array):
        
    # this list contains new translated descriptions
    tlds = []
    
    for desc in array:
        try:
            # Specifying the language for
            # detection
            # dbt : detection before translation
            dbt = detect(desc)

            # saving desc into text to translate
            if desc != 'Not Available':
                text = desc

                blob = TextBlob(text)

                # tat : text after translation
                tat = blob.translate(from_lang = detect(desc) , to = 'en')
                 # if description is already in english ---> save original description
                if dbt != 'en':
                    tlds.append(str(tat))

            # if not --> save translated description
            else:
                tlds.append(desc)
        except: tlds.append(desc)
        
    return tlds

In [6]:
def extract_categories(array):
    templist = [array[i][0]['name'] for i in range(len(array))]
    
    
    return templist
    

In [8]:
def add_photos(array):
    
    # list of links
    lol = []
    
    # index
    index = 0
    for fsq_id in array:
        
        try:
            
            url = f"https://api.foursquare.com/v3/places/{fsq_id}/photos?limit=1&sort=POPULAR&classifications=outdoor"

            headers = {
                "accept": "application/json",
                "Authorization": "fsq3bR9nCSrR/WbzD82rlvh990Q70wuc8BuuRs0Ypm6fx+w="
                    }

            response = requests.get(url, headers=headers).json()
            
            if response[0]['prefix'] != '':
                lol.append(response[0]['prefix']+'original'+response[0]['suffix'])
            else:
                lol.append('Not Available')
            print(response.text)
        
        except: print('photo error')
            
    return lol

In [43]:
def retrieve_places(a , c):

    """
    a : condition --- >  example : coffee , art gallery , etc ...
    c : city name
    """
    
    
    lat , long = get_latlong(c)
    if type(lat) != str:

        if a != '':
            fields_url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&query={a}&fields=fsq_id%2Cname%2Ctel%2Cprice%2Crating%2Cdescription%2Clocation%2Ccategories"

        else:
            fields_url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&fields=fsq_id%2Cname%2Ctel%2Cprice%2Crating%2Cdescription%2Clocation%2Ccategories%2Cfeatures"


        url = fields_url

        headers = {
            "Accept": "application/json",
            "Authorization": key,
            "Accept-Language": 'en'
        }

        response = requests.get(url, headers=headers)

        data = response.json()

        df = pd.json_normalize(data['results'])

        #deleting unnecessary columns

        try:
            df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel'
                                           ,'location.country', 'location.region'
                                           , 'description' , 'categories']),1,inplace=True)

        except: df = df
            
        
        
        """
        in this phase we add empty columns if columns are not available already
        
        ============== remember to change range when changing number of retrieved rows ==============
        """
        if 'price' not in df.columns:
            df.insert(len(df.columns) , 'price' , ['Not Available' for i in range(df.shape[0])] )
        
        if 'description' not in df.columns:
            df.insert(len(df.columns) , 'description' , ['Not Available' for i in range(df.shape[0])])
        
        if 'rating' not in df.columns:
            df.insert(len(df.columns) , 'rating' , ['Not Available' for i in range(df.shape[0])])

        
#         # renaming columns
        if 'location.country' in df.columns and 'location.region' in df.columns:
            df.rename(columns = {'location.country':'country' , 'location.region':'region'}, inplace = True)
    
        


        # filling nan values

        df = df.fillna('Not Available')


        # translating process starts here
        # error handling
        if 'description' in df.columns:

            # extracting
            array = df['description'].to_list()

            # tdl : translated descriptions list

            """
            in this line we call function to translate all descriptions as following

            other than english ---> translate

            Not Available ---> keep it as it is

            english description ---> keep it as it's
            """ 
            
            tdl = translate(array)

            # insert it into last 
            df.insert(df.columns.get_loc('description')+1  , 'translated_description' , tdl)

        if 'name' in df.columns:
            array_n = df['name'].to_list()
            
            tnl = translate(array_n)
            
            df.insert(df.columns.get_loc('name')+1 , 'translated_name' , tnl)

        
        if 'region' in df.columns:
            array_r = df['region'].to_list()
            
            trl = translate(array_r)
            
            df.insert(df.columns.get_loc('region')+1 , 'translated_region' , trl)

        if 'categories' in df.columns:
            print(df.categories)
            templist = df['categories'].to_list()
            templist = extract_categories(templist)
            
            df.drop(['categories'] , inplace = True , axis = 1)
            df.insert(len(df) , 'categories' , templist)
            
        try:
            # changing datatypes
            df = df.astype({'price': str , 'rating': str})
        except: df = df
        
        
        '''
        adding photos to dataframe 
        
        ==== i suggest to make this function separately rather th
        lol : ---> list of links
        '''
#         lol = add_photos(df['fsq_id'].to_list())
        
#         df.insert(len(df) , 'photo_url' , lol)
        
        data = df.to_json(orient = 'records')
        return df , data
    else:
        return lat , long 
    

In [44]:
'''

test field


'''

'\n\ntest field\n\n\n'

In [45]:
df, data_json = retrieve_places('' , 'cairo')
df

C:\Users\Abdullah\AppData\Local\Temp\ipykernel_21036\1004487.py:36: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel'


0    [{'id': 13035, 'name': 'Coffee Shop', 'icon': ...
1    [{'id': 13035, 'name': 'Coffee Shop', 'icon': ...
2    [{'id': 13035, 'name': 'Coffee Shop', 'icon': ...
3    [{'id': 13064, 'name': 'Pizzeria', 'icon': {'p...
4    [{'id': 13288, 'name': 'Kebab Restaurant', 'ic...
5    [{'id': 13012, 'name': 'Hookah Bar', 'icon': {...
6    [{'id': 13035, 'name': 'Coffee Shop', 'icon': ...
7    [{'id': 13034, 'name': 'Café', 'icon': {'prefi...
8    [{'id': 13031, 'name': 'Burger Joint', 'icon':...
9    [{'id': 13065, 'name': 'Restaurant', 'icon': {...
Name: categories, dtype: object


,fsq_id,name,translated_name,price,rating,country,region,translated_region,tel,description,categories,translated_description
0,60539559afff524f93bef5cb,ستاربكس,Starbox,1.0,8.7,EG,Muḩāfaz̧at al Qāhirah,Muḩāfaz̧at al Qāhirah,Not Available,Not Available,Coffee Shop,Not Available
1,56e3232bcd1044008992e42c,كوستا كوفى,Costa Coffee,1.0,7.9,EG,Muḩāfaz̧at al Qāhirah,Muḩāfaz̧at al Qāhirah,02 23955506,Not Available,Coffee Shop,Not Available
2,5ac227211f8ed65a8a965606,ستاربكس,Starbox,1.0,8.4,EG,Muḩāfaz̧at al Jīzah,Muḩāfaz̧at al jīzah,Not Available,Not Available,Coffee Shop,Not Available
3,4c924268b641236a82568879,بيتزا هت,Bitza Hat,1.0,7.3,EG,Muḩāfaz̧at al Qāhirah,Muḩāfaz̧at al Qāhirah,02 27941315,Not Available,Pizzeria,Not Available
4,56be0a92498e1a2f8abeda87,Sikh Mashwi (سيخ مشوي),Sikh Mushwi (سيخ مشوي),Not Available,8.2,EG,الجيزة,Giza,02 37605454,Not Available,Kebab Restaurant,Not Available
5,541cbaf9498e3d028fdcc628,قهوة الكراسي البيضاء,White chairs coffee,1.0,7.6,EG,Muḩāfaz̧at al Qāhirah,Muḩāfaz̧at al Qāhirah,Not Available,Not Available,Hookah Bar,Not Available
6,5e617321f04f090008ea176c,ستاربكس,Starbox,1.0,9.3,EG,Muḩāfaz̧at al Qāhirah,Muḩāfaz̧at al Qāhirah,Not Available,Not Available,Coffee Shop,Not Available
7,581b8fffbb5d611d3e42f3f6,Villa Garden City (الفيلا جاردن سيتي),Villa Garden City (Villa Garden City),1.0,7.4,EG,Muḩāfaz̧at al Qāhirah,Muḩāfaz̧at al Qāhirah,Not Available,Not Available,Café,Not Available
8,4bf2d2dae5eba59373d81d90,McDonald's (ماكدونالدز),McDonald's (McDonald's),1.0,8.0,EG,Muḩāfaz̧at al Jīzah,MuḩAFAZ̧at al jisah,02 33372226,Not Available,Burger Joint,Not Available
9,4c31702ea0ced13a76b7136e,تشيليز,Chelse,Not Available,7.2,EG,Muḩāfaz̧at al Qāhirah,Muḩāfaz̧at al Qāhirah,Not Available,Not Available,Restaurant,Not Available


In [17]:
df['photos']

0    https://fastly.4sqi.net/img/general/original/1...
1    https://fastly.4sqi.net/img/general/original/5...
2    https://fastly.4sqi.net/img/general/original/2...
3    https://fastly.4sqi.net/img/general/original/7...
4    https://fastly.4sqi.net/img/general/original/K...
5    https://fastly.4sqi.net/img/general/original/3...
6    https://fastly.4sqi.net/img/general/original/1...
7    https://fastly.4sqi.net/img/general/original/4...
8    https://fastly.4sqi.net/img/general/original/1...
9    https://fastly.4sqi.net/img/general/original/5...
Name: photos, dtype: object

In [29]:

    
# list of links
lol = []
headers = {
        "accept": "application/json",
        "Authorization": "fsq3bR9nCSrR/WbzD82rlvh990Q70wuc8BuuRs0Ypm6fx+w="
            }
# index
index = 0
for fsq_id in df['fsq_id']:


    url = f"https://api.foursquare.com/v3/places/{fsq_id}/photos?limit=1&sort=POPULAR&classifications=outdoor"
    
    response = requests.get(url, headers=headers).json()
    print(response)
    if response != [] :
        lol.append(response[0]['prefix']+'original'+response[0]['suffix'])
    else:
        lol.append('Not Available')
        



[]
[{'id': '50a14bd5e4b091ce32e3056b', 'created_at': '2012-11-12T19:19:49.000Z', 'prefix': 'https://fastly.4sqi.net/img/general/', 'suffix': '/565838_CxReRDTzLKE5NFopYQyliDGO3Pwtt4-FkrMLXGxQTDM.jpg', 'width': 612, 'height': 612, 'classifications': ['outdoor']}]
[]
[]
[]
[]
[]
[]
[]
[]


In [30]:
lol


['Not Available',
 'https://fastly.4sqi.net/img/general/original/565838_CxReRDTzLKE5NFopYQyliDGO3Pwtt4-FkrMLXGxQTDM.jpg',
 'Not Available',
 'Not Available',
 'Not Available',
 'Not Available',
 'Not Available',
 'Not Available',
 'Not Available',
 'Not Available']

In [14]:
app = Flask(__name__)


@app.route('/api' , methods = ['GET'])

def index():
    userInputb = str(request.args['query'])
    userInputa = str(request.args['query'])
    df, data_json = retrieve_places(userInputa , userInputb)

    return data_json


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Sep/2022 17:22:23] "GET / HTTP/1.1" 404 -
C:\Users\Abdullah\AppData\Local\Temp\ipykernel_34628\4038082940.py:35: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel'
127.0.0.1 - - [29/Sep/2022 17:22:54] "GET /api?query=riyadh HTTP/1.1" 200 -
